# Video Processing for BT Lab
This notebook extracts metadata from video files 

<br>

![UofC logo](./pictures/uofc_logo-black.jpg)

In [1]:
#import libraries
import os
import cv2
import json
import json
import datetime
import pandas as pd

In [2]:
repo_dir = os.getcwd()
json_dir = repo_dir + "/records/JSON"

In [3]:
def get_video_properties(mp4_file: str) -> list:
    try:
        video_properties = []
        video = cv2.VideoCapture(mp4_file)
        old_fps = video.get(cv2.CAP_PROP_FPS)
        number_of_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
        length = number_of_frames / old_fps
        video_time = datetime.timedelta(seconds=length)
        video_properties.append(number_of_frames)
        video_properties.append(old_fps)
        video_properties.append(length)
        video_properties.append(video_time)
    except Exception as e:
        print(f"{type(e)} - from {mp4_file}")
    
    return video_properties

In [4]:
def add_video_properties(metadata:dict) -> dict:
    new_patient_info = {}

    for patient_id, patient_info in metadata.items():
        try:
            video_path = patient_info["local path"]
            frames, fps, length_in_seconds, length_in_datetime = get_video_properties(video_path)
            patient_info["frames"] = frames
            patient_info["old fps"] = fps
            patient_info["length"] = length_in_seconds
            patient_info["length (datetime)"] = length_in_datetime
            new_patient_info[patient_id] = patient_info
        except Exception as e:
            print(f"{type(e)} - {patient_info['ID']}")

    return new_patient_info 

In [5]:
def load_json(json_dir:str, filename:str) -> dict:
    full_path = json_dir + "/" + filename

    with open(full_path, "r") as json_data:
        data = json.load(json_data)

    return(data)

In [6]:
def export_to_excel(patient_info:dict, filename: str):
    rgb_df = pd.DataFrame.from_dict(patient_info).transpose()
    rgb_df = rgb_df.drop(columns="local path")
    new_df = rgb_df.set_index("ID")
    new_df.to_excel(filename)
    return new_df

## Get video metadata and export to xlsx file (stage 2.1)

In [7]:
""" load JSON files """

metadata_rgb = load_json(json_dir, "rgb.json")
metadata_thermal = load_json(json_dir, "thermal.json")

In [8]:
""" extract metadata from video (rgb) """

complete_metadata_rgb = add_video_properties(metadata_rgb)
complete_metadata_rgb

{'0': {'ID': 'Arun',
  'distance': '2 Meters',
  'blanket': 'With Blankets',
  'breathing': 'Hold Breath',
  'filename': 'Arun2.mp4',
  'frames': 2146.0,
  'length': 73.054497,
  'old fps': 29.375330583687408,
  'local path': 'E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\\2 Meters\\With Blankets\\Hold Breath\\Arun2.mp4',
  'length (datetime)': datetime.timedelta(seconds=73, microseconds=54497)},
 '1': {'ID': 'Arun',
  'distance': '2 Meters',
  'blanket': 'With Blankets',
  'breathing': 'Hold Breath',
  'filename': 'Arun2_2.mp4',
  'frames': 1138.0,
  'length': 38.59782,
  'old fps': 29.483530417002825,
  'local path': 'E:/Christian/DI_centre_structured/DI_CAMERA_P3225/Final/Arun\\2 Meters\\With Blankets\\Hold Breath\\Arun2_2.mp4',
  'length (datetime)': datetime.timedelta(seconds=38, microseconds=597820)},
 '2': {'ID': 'Arun',
  'distance': '2 Meters',
  'blanket': 'With Blankets',
  'breathing': 'Relaxed',
  'filename': 'Arun2.mp4',
  'frames': 143.0,
  'length': 7.703

In [9]:
""" patient info to pandas DataFrame"""
rgb_xlsx = repo_dir + "/records/xlsx/rgb.xlsx"
rgb_df = export_to_excel(complete_metadata_rgb, rgb_xlsx)

In [10]:
rgb_df

,distance,blanket,breathing,filename,frames,length,old fps,length (datetime)
ID,,,,,,,,
Arun,2 Meters,With Blankets,Hold Breath,Arun2.mp4,2146.0,73.054497,29.375331,0 days 00:01:13.054497
Arun,2 Meters,With Blankets,Hold Breath,Arun2_2.mp4,1138.0,38.59782,29.48353,0 days 00:00:38.597820
Arun,2 Meters,With Blankets,Relaxed,Arun2.mp4,143.0,7.703175,18.563774,0 days 00:00:07.703175
Arun,2 Meters,With Blankets,Relaxed,Arun2_2.mp4,1137.0,38.190605,29.77172,0 days 00:00:38.190605
Arun,2 Meters,Without Blankets,Hold Breath,short.mp4,1790.0,36.113806,49.565532,0 days 00:00:36.113806
...,...,...,...,...,...,...,...,...
Sara,2 Meters,Without Blankets,Relaxed,Sara Relaxed.mp4,1497.0,24.849016,60.243834,0 days 00:00:24.849016
Sara,3 Meters,With Blankets,Hold Breath,Sara HB.mp4,1252.0,20.772879,60.270895,0 days 00:00:20.772879
Sara,3 Meters,With Blankets,Relaxed,Sara Relaxed.mp4,1222.0,20.266021,60.297974,0 days 00:00:20.266021


In [18]:
rgb_df["old fps"].max()

60.46067492664611